In [19]:
#!pip install -U keras-tuner

import pandas as pd
import numpy as np
from keras.datasets import boston_housing
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
from kerastuner.engine import hyperparameters
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import time
import math
from keras import utils
#from google.colab import drive
#drive.mount('/content/drive')

TRAIN_FILE_PATH = '/content/drive/My Drive/ИАД/train.csv'

# загружаем данные
x = pd.read_csv(TRAIN_FILE_PATH) 


# все буквы и цифры шифруются, чтобы остались только числа
df = pd.DataFrame(x)
x = df.apply(preprocessing.LabelEncoder().fit_transform)

x

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,...,X345,X346,X347,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,2466,32,23,17,0,3,24,9,14,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,1,366,32,21,19,4,3,28,11,14,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,2,69,20,24,34,2,3,27,9,23,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,3,133,20,21,34,5,3,27,11,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,4,106,20,23,34,5,3,12,3,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,4204,1657,8,20,16,2,3,0,3,16,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4205,4205,1766,31,16,40,3,3,0,7,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4206,4206,1801,8,23,38,0,3,0,6,4,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4207,4207,280,9,19,25,5,3,0,11,20,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [20]:
# функция извлекает правильные значение из тренировочных данных
def get_truth(array):
  data = []
  for col_name, col_info in array.items():
    # находится столбец y
    if col_name == "y":
      # все данные из него пререписываются в массив
      for line_number, line_info in col_info.items():
        temp_array = []
        temp_array.append(line_info)
        data.append(temp_array)
  # массив трансформируется в numpy и меняется его размерность
  return np.array(data, dtype=np.float64).reshape(array.shape[0], 1)


y = get_truth(x)
y

array([[2466.],
       [ 366.],
       [  69.],
       ...,
       [1801.],
       [ 280.],
       [1921.]])

In [21]:
# столбцы 'y' и 'ID' не нужны при работе больше
x.drop('y', axis='columns', inplace=True)
x.drop('ID', axis='columns', inplace=True)
x

,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,X42,...,X345,X346,X347,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,32,23,17,0,3,24,9,14,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,32,21,19,4,3,28,11,14,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,20,24,34,2,3,27,9,23,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,20,21,34,5,3,27,11,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,20,23,34,5,3,12,3,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,8,20,16,2,3,0,3,16,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4205,31,16,40,3,3,0,7,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4206,8,23,38,0,3,0,6,4,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4207,9,19,25,5,3,0,11,20,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [22]:
# функция нормализуем данные для работы
def normalize_data(data):
  mean = data.mean(axis=0)
  std = data.std(axis=0)
  data -= mean
  data /= std
  return data

  
x_norm = normalize_data(x)
x_norm

,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,X42,...,X345,X346,X347,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0.162993,1.393322,-0.028118,-1.678071,0.028935,1.291964,0.751698,0.339405,-0.116108,NaN,-0.284872,4.03065,-0.865143,-0.021801,-0.051183,-0.087517,11.247906,-0.332457,-0.408087,19.533195,-0.30857,-0.145263,-0.043633,-0.070803,-1.466267,-0.183402,-0.211955,-0.067331,1.817387,-0.106254,-0.015414,-0.074116,1.817387,-0.067331,1.817387,-0.185468,-0.015414,-0.026704,-0.107392,-0.015414,...,-0.151122,-0.223329,NaN,-4.237357,-0.216803,-0.715734,-0.650246,-0.239287,-0.046286,1.981827,-0.783412,-0.468232,-0.034483,-0.863046,-0.181316,-0.287786,0.187514,-1.041363,-1.749861,-0.053465,-0.053465,-0.034483,-0.233688,-0.258659,-0.021801,-0.081825,-0.120241,-0.021801,-0.140062,-0.542412,-0.684086,-0.246417,1.475156,-0.145263,-0.097941,-0.090232,-0.087517,-0.04081,-0.021801,-0.037778
1,0.162993,1.158883,0.155369,0.620896,0.028935,1.776763,1.437340,0.339405,-0.116108,NaN,-0.284872,-0.24804,-0.865143,-0.021801,-0.051183,-0.087517,11.247906,-0.332457,-0.408087,-0.051183,-0.30857,-0.145263,-0.043633,-0.070803,0.681842,-0.183402,-0.211955,-0.067331,1.817387,-0.106254,-0.015414,-0.074116,1.817387,-0.067331,1.817387,-0.185468,-0.015414,-0.026704,-0.107392,-0.015414,...,-0.151122,-0.223329,NaN,0.235940,-0.216803,-0.715734,-0.650246,-0.239287,-0.046286,-0.504465,-0.783412,-0.468232,-0.034483,-0.863046,-0.181316,-0.287786,0.187514,-1.041363,0.571338,-0.053465,-0.053465,-0.034483,-0.233688,-0.258659,-0.021801,-0.081825,-0.120241,-0.021801,-0.140062,-0.542412,1.461457,-0.246417,-0.677733,-0.145263,-0.097941,-0.090232,-0.087517,-0.04081,-0.021801,-0.037778
2,-0.710475,1.510542,1.531527,-0.528587,0.028935,1.655563,0.751698,1.618197,-0.116108,NaN,-0.284872,-0.24804,-0.865143,-0.021801,-0.051183,11.423670,-0.088884,-0.332457,-0.408087,-0.051183,-0.30857,-0.145263,-0.043633,-0.070803,0.681842,5.451199,4.716869,-0.067331,1.817387,-0.106254,-0.015414,-0.074116,1.817387,-0.067331,1.817387,-0.185468,-0.015414,-0.026704,-0.107392,-0.015414,...,-0.151122,-0.223329,NaN,0.235940,-0.216803,1.396835,-0.650246,-0.239287,-0.046286,1.981827,-0.783412,-0.468232,-0.034483,1.158412,-0.181316,-0.287786,0.187514,-1.041363,0.571338,-0.053465,-0.053465,-0.034483,-0.233688,-0.258659,-0.021801,-0.081825,-0.120241,-0.021801,-0.140062,-0.542412,-0.684086,-0.246417,-0.677733,-0.145263,-0.097941,-0.090232,11.423670,-0.04081,-0.021801,-0.037778
3,-0.710475,1.158883,1.531527,1.195637,0.028935,1.655563,1.437340,-1.081476,-0.116108,NaN,-0.284872,-0.24804,-0.865143,-0.021801,-0.051183,-0.087517,-0.088884,-0.332457,-0.408087,-0.051183,-0.30857,-0.145263,-0.043633,-0.070803,0.681842,5.451199,4.716869,-0.067331,1.817387,-0.106254,-0.015414,-0.074116,1.817387,-0.067331,1.817387,-0.185468,-0.015414,-0.026704,-0.107392,-0.015414,...,-0.151122,-0.223329,NaN,0.235940,-0.216803,1.396835,-0.650246,-0.239287,-0.046286,-0.504465,-0.783412,-0.468232,-0.034483,1.158412,-0.181316,-0.287786,0.187514,-1.041363,0.571338,-0.053465,-0.053465,-0.034483,-0.233688,-0.258659,-0.021801,-0.081825,-0.120241,45.858481,-0.140062,-0.542412,-0.684086,-0.246417,-0.677733,-0.145263,-0.097941,-0.090232,-0.087517,-0.04081,-0.021801,-0.037778
4,-0.710475,1.393322,1.531527,1.195637,0.028935,-0.162435,-1.305229,0.197316,-0.116108,NaN,-0.284872,-0.24804,-0.865143,-0.021801,-0.051183,-0.087517,-0.088884,-0.332457,-0.408087,-0.051183,-0.30857,-0.145263,-0.043633,-0.070803,0.681842,5.451199,4.716869,-0.067331,1.817387,-0.106254,-0.015414,-0.074116,1.817387,-0.067331,1.817387,-0.185468,-0.015414,-0.026704,-0.107392,-0.015414,...,-0.151122,-0.223329,NaN,0.235940,-0.216803,1.396835,-0.650246,-0.239287,-0.046286,-0.504465,-0.783412,-0.468232,-0.034483,1.158412,-0.181316,-0.287786,0.187

In [23]:
def check_nans(data):
  # массив названий столбцов, в которых одни NaN
  nan_cols = []
  for col_name, col_info in data.items():
    for line_number, line_info in col_info.items():
      if math.isnan(line_info):
        print(f"{col_name } is NaNs")
        nan_cols.append(col_name)
        break
  if nan_cols == []:
    print("[INFO] THERE IS NO NaNS IN DATA!")
  return nan_cols

nan_cols = check_nans(x_norm)
type(x_norm)

X11 is NaNs
X93 is NaNs
X107 is NaNs
X233 is NaNs
X235 is NaNs
X268 is NaNs
X289 is NaNs
X290 is NaNs
X293 is NaNs
X297 is NaNs
X330 is NaNs
X347 is NaNs


pandas.core.frame.DataFrame

In [24]:
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
x_norm = imp.fit_transform(x_norm)
print(x_norm)
print(type(x_norm))

[[ 0.16299273  1.39332233 -0.02811821 ... -0.04081026 -0.02180104
  -0.03777847]
 [ 0.16299273  1.15888324  0.15536947 ... -0.04081026 -0.02180104
  -0.03777847]
 [-0.71047536  1.51054187  1.53152706 ... -0.04081026 -0.02180104
  -0.03777847]
 ...
 [-1.58394344  1.39332233  1.89850241 ... -0.04081026 -0.02180104
  -0.03777847]
 [-1.51115443  0.92444416  0.7058325  ... -0.04081026 -0.02180104
  -0.03777847]
 [ 1.18203883  0.92444416 -1.31253196 ... -0.04081026 -0.02180104
  -0.03777847]]
<class 'numpy.ndarray'>


In [25]:
# делим в соотношении 70 на 30
(x_train, x_test) = train_test_split(x_norm, test_size=0.3)
(y_train, y_test) = train_test_split(y, test_size=0.3)

In [26]:
# функция принимает данные, прочитанные из csv файла, переводит их в numpy_array
def convert_csv_to_numpy(array):
  array_to_convert = []
  for col_name, col_info in array.items():
    # временный массив создается для каждого СТОЛБЦА
    temp_array = []
    for line_number, line_info in col_info.items():
        temp_array.append(line_info)
    # после прохождения всего столбца временный массив добавляется в основной
    if temp_array != []:
      array_to_convert.append(temp_array)
  # в самом конце основном массив конвертируется в numpy_array
  return np.array(array_to_convert, dtype=np.float64)

x_train

array([[ 0.09020372, -0.95106851,  0.88932018, ..., -0.04081026,
        -0.02180104, -0.03777847],
       [ 0.09020372, -0.95106851, -0.30334973, ..., -0.04081026,
        -0.02180104, -0.03777847],
       [-0.34653032,  0.80722462,  1.43978322, ..., -0.04081026,
        -0.02180104, -0.03777847],
       ...,
       [-0.78326436,  0.92444416, -0.11986205, ..., -0.04081026,
        -0.02180104, -0.03777847],
       [ 0.38135975,  1.15888324, -0.85381276, ..., -0.04081026,
        -0.02180104, -0.03777847],
       [-1.58394344,  1.39332233,  1.89850241, ..., -0.04081026,
        -0.02180104, -0.03777847]])

In [27]:
x_test

array([[-1.21999841,  1.0416637 , -0.11986205, ..., -0.04081026,
        -0.02180104, -0.03777847],
       [ 0.38135975,  1.0416637 , -1.31253196, ..., -0.04081026,
        -0.02180104, -0.03777847],
       [ 0.67251578, -1.18550759, -0.9455566 , ..., -0.04081026,
        -0.02180104, -0.03777847],
       ...,
       [ 1.18203883,  0.92444416, -1.31253196, ..., -0.04081026,
        -0.02180104, -0.03777847],
       [ 0.23578173,  1.27610278, -1.31253196, ..., -0.04081026,
        -0.02180104, -0.03777847],
       [ 1.03646081, -0.95106851,  1.43978322, ..., -0.04081026,
        -0.02180104, -0.03777847]])

In [28]:
# меняю формы обоих x, чтобы первое измерение совпадало с y
x_train = np.reshape(x_train, (x_train.shape[1], x_train.shape[0]))
x_test = np.reshape(x_test, (x_test.shape[1], x_test.shape[0]))

print(f"Y_TRAIN SHAPE IS {y_train.shape}")
print(f"Y_TEST SHAPE IS {y_test.shape}")
print(f"X_TRAIN SHAPE IS {x_train.shape}")
print(f"X_TEST SHAPE IS {x_test.shape}")
print("\n\n")

Y_TRAIN SHAPE IS (2946, 1)
Y_TEST SHAPE IS (1263, 1)
X_TRAIN SHAPE IS (364, 2946)
X_TEST SHAPE IS (364, 1263)





In [29]:
model = Sequential()
model.add(Dense(1024, activation="relu", input_shape=(x_train.shape[1],)))
model.add(Dense(512, activation="relu"))
model.add(Dense(256, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(16, activation="relu"))
model.add(Dense(8, activation="relu"))
model.add(Dense(4, activation="relu"))
model.add(Dense(2, activation="relu"))
model.add(Dense(1))
print("[INFO] NN has been created")
model.compile(optimizer="adam", loss="mse", metrics=["mae"])
print("[INFO] NN has been compiled")
model.summary()
# начинаем обучение нейронной сети
model.fit(x_train, y_train, epochs=150, batch_size=1000, verbose=1, validation_split=0.3)
print("[INFO] Training has been finished")

[INFO] NN has been created
[INFO] NN has been compiled
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1024)              3017728   
_________________________________________________________________
dense_6 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_7 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_8 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_9 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_10 (Dense)             (None, 32)                2080      
_________________________________________________________________

In [30]:
print("[INFO] RUNNING ON TEST DATA: \n")
# вычисляем ошибки и выводим их на экран
mse, mae = model.evaluate(x_test, y_test, verbose=1)
print(f"[INFO] Mean squared error is {mse}")
print(f"[INFO] Mean absolute error is {mae}")

[INFO] RUNNING ON TEST DATA: 



ValueError: ignored

In [31]:
# функция создание нейронной сети
def build_NN(hp):
  model = Sequential()
  # варианты активационых функций
  activation_choice = hp.Choice("activation", values=["relu", "sigmoid", "tanh", "elu", "selu"])
  # создаем один входной слой
  model.add(Dense(units=hp.Int("units_input", min_value=128,max_value=1024, step=32), activation=activation_choice, input_dim=364))
  # опционально добавляются еще слои разной конфигурации
  for i in range(hp.Int("n_layers", 2, 5)):
    model.add(Dense(units=hp.Int("units_"+str(i), min_value=128, max_value=1024, step=32), activation=activation_choice))
  # создаем один выходной слой
  model.add(Dense(1, activation="softmax"))
  # компиляция сети происходит с выборо одного из оптимизаторов
  model.compile(optimizer=hp.Choice("optimizer", values=["adam", "rmsprop", "SGD"]), loss="categorical_crossentropy", metrics=["accuracy"])
  model.summary()
  return model


def find_best_NN(x_train_norm, y_train):
  # создаю тюнер, который сможет подобрать оптимальную архитектуру модели
  tuner = RandomSearch(build_NN, objective="val_accuracy", max_trials=20, executions_per_trial=1)
  # общие данные выводятся на экран
  print("\n\n\nSEARCH SPACE SUMMARY")
  tuner.search_space_summary()
  print("\n\n\n")
  # начинается автоматический подбор гиперпараметров
  print('[INFO] start searching')
  tuner.search(x_train, y_train, batch_size=256, epochs=3, validation_split=0.3, verbose=1)
  # выбираем лучшую модель
  print("\n\n\nRESULTS SUMMARY")
  tuner.results_summary()
  print("\n\n\n")
  # получаем лучшую модель
  print("\n\n\nHERE IS THE BEST MODEL\n\n\n")
  best_model = tuner.get_best_models(num_models=1)[0]

  return best_model
  

best_model = find_best_NN(x_train, y_train)

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               46720     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 79,873
Trainable params: 79,873
Non-trainable params: 0
_________________________________________________________________



SEARCH SPACE SUMMARY






[INFO] start searching
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 960)               350400    
_________________________________________________________________
dense_1 (Dense)              (None, 672)               645792    
_________________________________________________________________
dense_2 (Dense)              (None, 832)               559936    
_________________________________________________________________
dense_3 (Dense)              (None, 128)               106624    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 1,662,881
Trainable params: 1,662,881
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3


ValueError: ignored

In [32]:
best_model.fit(x_train, y_train, epochs=50, batch_size=100, verbose=1)

NameError: ignored

In [33]:
best_model.evaluate(x_test, y_test, verbose=1)

NameError: ignored